# LabOP labware ontology Explorer

This notebook is a work in progress. It is intended to be a place to explore the LabOP labware ontology.

The ontology is a set of OWL ontologies that describe the structure and semantics of labware. 

The ontology is designed to be used in conjunction with the [Labware Ontology Ontology](www.github/bioprotocols/)

In [ ]:
#  currently only reflib 4.2.1 is supported
#! pip install rdflib==4.2.1
#! pip install packaging==21.0
import os
import pathlib
import logging
import pandas as pd
import labop_labware_ontology.labop_labware_ontology_impl as lolwi

In [ ]:
lwi = lolwi.LabwareInterface()
lwi.prefix_dict

In [ ]:
list(lwi.lolw_tbox.lolwt.classes())

In [ ]:
lwi.lolw_abox.import_csv('../labware_catalogues/labop_labware_catalog.csv')

In [ ]:
lwi.lolw_abox.lolwa.mtp_96_flat_uv.is_a

In [ ]:
lwi.lolw_abox.lolwa.mtp_96_flat_uv.hasNumCols, lwi.lolw_abox.lolwa.mtp_96_flat_uv.hasNumWells

In [ ]:
lwi.lolw_abox.lolwa["mtp_384_v_clear_clear"].hasLength.length

In [ ]:
res = lwi.lolw_abox.lolwa.search(hasManifacturer = "Corning", hasProductID = "3574")
res

In [ ]:
Manufacturer = "Corning"
ProductID = "781280"

res = lwi.lolw_abox.lolwa.search(hasManufacturer=Manufacturer) #, hasProductID=ProductID)
res

In [ ]:
lwi.lolw_abox.lolwa.mtp_384_flat_white_white_nbs.hasProductID

In [ ]:
for lw in res:
    print(lw.hasProductID)

In [ ]:
Manufacturer = "Corning"
ProductID = "3574"

res = lwi.lolw_abox.lolwa.search(hasManufacturer=Manufacturer, hasProductID=ProductID)[0]
res.hasWidth.length

In [ ]:
lwi.export_ontologies(path='../ontologies/')

## Constructing the ontology by ttl files

In [ ]:
from ontopy import World
emmo_world = World()

base_iri = 'http://www.labop.org/labware-a#'
#lolwa_version_iri = f'http://www.labop.org/{__version__}/labware-a'

lw_abox_filename = "../ontologies/labop_labware_abox.ttl"

print("LOLabwareABox:lw_abox_filename:", lw_abox_filename)

print("loading ontology from file: ", lw_abox_filename, " ...")
lolwa = emmo_world.get_ontology(lw_abox_filename).load()

print(" ##### classes:", list(lolwa.classes()))  

In [ ]:
lwi = lolwi.LabwareInterface( emmo_filename='../ontologies/labop_labware_ontology.ttl', lw_tbox_filename='../ontologies/labop_labware_tbox.ttl', lw_abox_filename='../ontologies/labop_labware_abox.ttl')
lwi.prefix_dict

In [ ]:
list(lwi.lolw_abox.lolwa.classes())

In [ ]:
lw_abox = lwi.lolw_abox.lolwa

In [ ]:
list(lwi.lolw_tbox.lolwt.classes())

In [ ]:
list(lw_abox.classes())

In [ ]:
Manufacturer = "Corning"
ProductID = "781280"

res = lw_abox.search(hasManufacturer=Manufacturer) #, hasProductID=ProductID)
res
#print(res)

## Competency questions to be answered by the ontology

In [ ]:
# provide width, length, height of a plate instance 

# in which unit is the width, length, height defined?


# which force is required to open the lid of a screw cap vial ?

# is a certain plate of type "SLAS1-2004compliant" ?

# Where can I punch the septum of a screw cap vial / GC- / HPLC vial ?

# What is the max. volume of a well of a multiwell plate ?

# at with height can a robot grab a microtiterplate with a lid ?


## SPARQL queries

In [ ]:
prefix_dict = {
    'rdf': "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    'rdfs': "http://www.w3.org/2000/01/rdf-schema#",
    'xml': "http://www.w3.org/XML/1998/namespace",
    'xsd': "http://www.w3.org/2001/XMLSchema#",
    'owl': "http://www.w3.org/2002/07/owl#",
    'skos': "http://www.w3.org/2004/02/skos/core#",
    'dc': "http://purl.org/dc/elements/1.1/",
    'dcterm': "http://purl.org/dc/terms/",
    'dctype': "http://purl.org/dc/dcmitype/",
    'foaf': "http://xmlns.com/foaf/0.1/",
    'wd': "http://www.wikidata.org/entity/",
    'ex': "http://www.example.com/",
    'emmo': "http://emmo.info/emmo#",
    'lolw': "http://www.labop.org/labware'#",
}

In [ ]:
graph = lw.emmo_world.as_rdflib_graph()

for prefix, iri in prefix_dict.items():
    print(prefix, "--- ", iri )
    graph.bind(prefix, iri)

In [ ]:
# get all labware from Greiner

query = """

PREFIX lolw: <http://www.labop.org/labware#>

SELECT ?lm
WHERE {
    ?lm rdf:type lolw:Labware.
    ?lm lolw:hasManifacturer "Corning".
    }
"""

In [ ]:
results = list(lw.emmo_world.sparql(query))
results

In [ ]:
query = """

PREFIX lolw: <http://www.labop.org/labware#>

SELECT ?lw_name ?lw_num_wells 
WHERE {
    ?lw_name rdf:type lolw:Labware.
    ?lw_name lolw:hasManifacturer "Greiner".
    ?lw_name lolw:hasNumWells ?lw_num_wells.
    }
"""

In [ ]:
select_statement = re.findall(r"SELECT.*\n", query)[0]
variable_names = re.findall(r"\?(\w+)", select_statement)
variable_names

In [ ]:


# pandas dataframe from res_list with header df_head


res_df = pd.DataFrame(columns=variable_names, data=list(lw.emmo_world.sparql(query)))
#res_df.to_json()
#results_json = res_df.to_json(orient="records")